#Preparing Weather Data#
In this notebook we take the raw weather data from 1961 - 2019 and prepare it to be usable later on


-  importing pandas and our shared drive

In [ ]:
import pandas as pd
from IPython.core.debugger import set_trace
from google.colab import drive
drive.mount("/content/drive")
pd.set_option('display.max_rows', None)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


The problem with the weather data was, that the file containing all the valuable features had only the codes of the operating weather statoin for each row. We loaded this data into a dataframe 'df'.

A second file had the list of all weather stations and their corresponding statecodes (which we need to match our fire data). We loaded those into 'df2'. 

In [ ]:
df = pd.DataFrame()
df = df.append(pd.read_csv(f'/content/drive/Shared drives/BNCS411_Final_Project/Original Files/weather_without_state.csv', delimiter=","))
df2 = pd.DataFrame()
df2 = df2.append(pd.read_csv(f'/content/drive/Shared drives/BNCS411_Final_Project/weather_stations_codes.csv', delimiter=";"))

The first step is to cut out all rows prior to the 01.11.2000 as this is the date our fire data starts. 

In [ ]:
df['Data'] = pd.to_datetime(df['Data'])
df[~df['Data'].isin(pd.date_range(start='19610101', end='20001001'))]

We extract the two-letter statecode for each weather station from our df2 into a new column 'statecode' as they were not listed as such in their own columns but rather as eg. 'INDAIAL - SC'.

In [ ]:
df2["statecode"] = ""
for i in df2.index:
  df2['statecode'][i] = str(df2['Nome'][i])[-2:]
df2

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Nome,Código,Latitude,Longitude,Altitude,Status da Operação,Ínicio da Operação,statecode
0,ACARAU - CE,82294,-2.88,-40.14,16.50,Estação Desativada,01/01/1923,CE
1,AGUA BRANCA - AL,82989,-9.28,-37.90,605.34,Estação Operante,05/05/1928,AL
2,AIMORES - MG,83595,-19.49,-41.07,82.74,Estação Operante,01/06/1972,MG
3,ALAGOINHAS - BA,83249,-12.14,-38.42,130.92,Estação Operante,01/01/1931,BA
4,ALTAMIRA - PA,82353,-3.21,-52.21,74.04,Estação Operante,24/04/1927,PA
5,ALTO PARNAIBA - MA,82970,-9.10,-45.93,285.05,Estação Operante,21/08/1976,MA
6,APODI - RN,82590,-5.61,-37.81,150.00,Estação Operante,18/12/1962,RN
7,ARACAJU - SE,83096,-10.95,-37.04,4.72,Estação Operante,01/03/1910,SE
8,ARACUAI - MG,83442,-16.83,-42.05,289.00,Estação Operante,06/09/1918,MG
9,ARAGARCAS - GO,83368,-15.90,-52.23,345.00,Estação Operante,19/07/1970,GO


In this part the translation of the statecodes from out df2 to df happens. 'df' has around 5 Million rows, therefore we decided to use apply() as it is way faster than iterating over each row manually or by using iterrows(). 

**get_match()** 

returns us a list of NaN values except for the statecode matching our weatherstation code we gave it as a parameter. It iterates over all 265 weatherstations 'row' and looks to match the variable 'x', which is the weatherstation code in question.

**get_code()** 

gets executed once per row in our 'df', which is around 5 Million times. As a parameter it takes the Weatherstation code from the row at each iteration. It executes get_match() and receives the list, containing the matching statecode. Next, it drops all values but the matching statecode and returns that value to be written onto a column 'statec' on our original 'df'

In [ ]:
pd.set_option('display.max_rows', None)
def get_match(row, x):
  if str(row[1]) == str(x):

    return str(row[7])
  else:
    return

def get_code(x):
  statec = df2.apply(get_match, x=x, axis=1)
  if statec is not None:
    temp = pd.DataFrame(statec).dropna()
  return temp.iloc[0][0]

df['statec'] = df.Estacao.apply(get_code)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


The modified df gets saved to a .csv file and saved to the drive.

In [ ]:
df.to_csv('weather_with_state.csv')
!cp weather_with_state.csv "/content/drive/Shared drives/BNCS411_Final_Project"